### Assignment

(a) Construct a dataset with $N=20$ samples, following the model
$$
y_n = \sum_{p=0}^{d-1} \theta_p L_p(x_n) + e_n\,
$$
where $\theta_0=1$, $\theta_1=0.5$, $\theta_2=1.5$, $\theta_4=1$, for $-1<x<1$. Here, $L_p(x)$ is the Legendre polynomial of the pth order. The $N=20$ samples are random uniformly sampled from the interval $[-1,1]$. The noise samples $e_n$ are i.i.d. Gaussian with variance $\sigma^2=0.25^2$. Plot the dataset using Python command $\textrm{scatter}$.

(b) Run the regression using the same model where $d=5$, without any regularization. Plot the predicted curve and overlay with the training samples.

(c) Repeat (b) by running the regression with $d=20$. Explain your observations.

(d) Increase the number of training samples $N$ to $N=50$, $N=500$, $N=5000$, and repeat (c). Explain your observations.

(e) Construct a testing dataset with $M=1000$ testing samples. For each of the regression models trained in (b)-(d), compute the testing error.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import legendre
from scipy import stats
from IPython.display import display, Math

In [ ]:
class GroundTruth:
    """Ground truth function using Legendre polynomials"""
    def __init__(self, theta):
        self.d = len(theta)
        self.theta = theta
        self.L_vec = [legendre(i) for i in range(self.d)]

    def f(self, x):
        f_values = self.theta[0] * self.L_vec[0](x)
        for i in range(1, self.d):
            f_values += self.theta[i] * self.L_vec[i](x)
        return f_values

    def plot_samples_and_truth(self, x, y, figsize=(8, 5)):
        plt.figure(figsize=figsize)
        plt.scatter(x, y, color='blue', label='$y_n$', alpha=0.7)
        x_curve = np.linspace(-1, 1, 200)
        plt.plot(x_curve, self.f(x_curve), color='green', label='$f(x)$', linewidth=2)
        plt.xlabel('$x$', fontsize=12)
        plt.ylabel('$y$', fontsize=12)
        plt.legend()
        plt.title('Ground Truth and Samples')
        plt.grid(True, alpha=0.3)
        plt.show()


class LegendreRegression:
    """Legendre polynomial regression"""
    def __init__(self, d=5):
        self.d = d
        self.L_vec = [legendre(i) for i in range(d)]
        self.theta = np.zeros(d)

    def fit(self, x, y):
        X = np.column_stack([self.L_vec[i](x) for i in range(self.d)])
        self.theta = np.linalg.inv(X.T @ X) @ X.T @ y
        return self

    def predict(self, x):
        y_hat = self.theta[0] * self.L_vec[0](x)
        for i in range(1, self.d):
            y_hat += self.theta[i] * self.L_vec[i](x)
        return y_hat


def generate_samples(f, N, mu=0, std=0.25):
    """Generate N samples with Gaussian noise"""
    x = np.random.uniform(-1, 1, N)
    y = f(x) + np.random.normal(mu, std, N)
    return x, y


def plot_truth_and_model(gt, model, x, y, figsize=(8, 5)):
    """Plot ground truth, model prediction, and training data"""
    plt.figure(figsize=figsize)
    plt.scatter(x, y, color='blue', label='Training data', alpha=0.7, zorder=5)
    x_curve = np.linspace(-1, 1, 200)
    plt.plot(x_curve, gt.f(x_curve), color='green', label='True $f(x)$', linewidth=2)
    plt.plot(x_curve, model.predict(x_curve), color='red', linestyle='--', 
             label=f'Fitted model (d={model.d})', linewidth=2)
    plt.xlabel('$x$', fontsize=12)
    plt.ylabel('$y$', fontsize=12)
    plt.legend()
    plt.title(f'Legendre Regression (d={model.d}, N={len(x)})', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# Parameters
# Note: According to assignment: θ₀=1, θ₁=0.5, θ₂=1.5, θ₄=1
# For d=5, this means: θ₀=1, θ₁=0.5, θ₂=1.5, θ₃=0, θ₄=1
d = 5
theta = np.array([1, 0.5, 1.5, 0, 1])
mu = 0
std = 0.25
N = 20

# Set random seed for reproducibility
np.random.seed(42)

# Create ground truth and model
gt_d5 = GroundTruth(theta)
model_d5 = LegendreRegression(d)

print(f"True coefficients: {theta}")
print(f"Noise std: σ = {std}")
print(f"Noise variance: σ² = {std**2}")

## Part (a): Generate and plot dataset with N=20 samples

In [ ]:
# Generate samples
x_N20, y_N20 = generate_samples(gt_d5.f, N=20, mu=mu, std=std)

# Plot
gt_d5.plot_samples_and_truth(x_N20, y_N20)

## Part (b): Regression with d=5, N=20

In [ ]:
# Fit and plot
model_d5.fit(x_N20, y_N20)
plot_truth_and_model(gt_d5, model_d5, x_N20, y_N20)

# Calculate training error
y_train_pred = model_d5.predict(x_N20)
train_error_d5_N20 = np.mean((y_N20 - y_train_pred) ** 2)
print(f"Training MSE (d=5, N=20): {train_error_d5_N20:.6f}")
print(f"Theoretical training error σ²(1-d/N): {std**2 * (1 - d/N):.6f}")
print(f"Learned coefficients θ: {model_d5.theta}")
print(f"True coefficients: {theta}")

## Part (c): Regression with d=20, N=20

In [ ]:
model_d20_N20 = LegendreRegression(d=20)
model_d20_N20.fit(x_N20, y_N20)
plot_truth_and_model(gt_d5, model_d20_N20, x_N20, y_N20)

# Calculate training error
y_train_pred = model_d20_N20.predict(x_N20)
train_error_d20_N20 = np.mean((y_N20 - y_train_pred) ** 2)
print(f"Training MSE (d=20, N=20): {train_error_d20_N20:.6f}")
print(f"Theoretical training error σ²(1-d/N): {std**2 * (1 - 20/20):.6f} (should be ~0)")
print(f"Norm of coefficients: {np.linalg.norm(model_d20_N20.theta):.6f}")

**Observations for part (c):**

When $d=20$ and $N=20$, we have a **deterministic system** ($d = N$). This leads to:

1. **Perfect fit on training data**: The model can exactly interpolate all training points, resulting in near-zero training error.
2. **Overfitting**: The model "memorizes" the noise in the training data rather than learning the underlying pattern.
3. **Poor generalization**: The predicted curve shows wild oscillations between training points, indicating high variance.
4. **Large coefficients**: The norm of coefficients is typically very large, making the model sensitive to small changes in the data.

This is a classic example of the **bias-variance tradeoff**: we've reduced bias to near zero but increased variance dramatically.

## Part (d): Regression with d=20 and varying N

In [ ]:
# Generate datasets with different N
x_N50, y_N50 = generate_samples(gt_d5.f, N=50, mu=mu, std=std)
x_N500, y_N500 = generate_samples(gt_d5.f, N=500, mu=mu, std=std)
x_N5000, y_N5000 = generate_samples(gt_d5.f, N=5000, mu=mu, std=std)

In [ ]:
# N = 50
model_d20_N50 = LegendreRegression(d=20)
model_d20_N50.fit(x_N50, y_N50)
plot_truth_and_model(gt_d5, model_d20_N50, x_N50, y_N50)

y_train_pred = model_d20_N50.predict(x_N50)
train_error_d20_N50 = np.mean((y_N50 - y_train_pred) ** 2)
print(f"Training MSE (d=20, N=50): {train_error_d20_N50:.6f}")
print(f"Theoretical training error σ²(1-d/N): {std**2 * (1 - 20/50):.6f}")

In [ ]:
# N = 500
model_d20_N500 = LegendreRegression(d=20)
model_d20_N500.fit(x_N500, y_N500)
plot_truth_and_model(gt_d5, model_d20_N500, x_N500, y_N500)

y_train_pred = model_d20_N500.predict(x_N500)
train_error_d20_N500 = np.mean((y_N500 - y_train_pred) ** 2)
print(f"Training MSE (d=20, N=500): {train_error_d20_N500:.6f}")
print(f"Theoretical training error σ²(1-d/N): {std**2 * (1 - 20/500):.6f}")

In [ ]:
# N = 5000
model_d20_N5000 = LegendreRegression(d=20)
model_d20_N5000.fit(x_N5000, y_N5000)
plot_truth_and_model(gt_d5, model_d20_N5000, x_N5000, y_N5000)

y_train_pred = model_d20_N5000.predict(x_N5000)
train_error_d20_N5000 = np.mean((y_N5000 - y_train_pred) ** 2)
print(f"Training MSE (d=20, N=5000): {train_error_d20_N5000:.6f}")
print(f"Theoretical training error σ²(1-d/N): {std**2 * (1 - 20/5000):.6f}")

**Observations for part (d):**

As $N$ increases while keeping $d=20$ fixed:

1. **Ratio $d/N$ decreases**: The system becomes increasingly overdetermined ($N \gg d$).
2. **Overfitting reduces**: With more data, the model has less opportunity to "memorize" noise.
3. **Training error increases toward $\sigma^2$**: Expected training error $\approx \sigma^2(1 - d/N) \to \sigma^2$ as $N \to \infty$.
4. **Model approaches true function**: The learned coefficients become more accurate.
5. **Coefficient norm decreases**: The model becomes more stable.

| N | $d/N$ | Expected Training Error |
|---|-------|------------------------|
| 20 | 1.0 | 0 |
| 50 | 0.4 | 0.0375 |
| 500 | 0.04 | 0.06 |
| 5000 | 0.004 | 0.0622 |

## Part (e): Testing Error with M=1000 test samples

In [ ]:
# Generate test dataset
M = 1000
x_test, y_test = generate_samples(gt_d5.f, N=M, mu=mu, std=std)

# Compute test error for all models
models = [
    (model_d5, "d=5, N=20"),
    (model_d20_N20, "d=20, N=20"),
    (model_d20_N50, "d=20, N=50"),
    (model_d20_N500, "d=20, N=500"),
    (model_d20_N5000, "d=20, N=5000"),
]

print("=" * 50)
print("Testing Errors (M=1000 test samples)")
print("=" * 50)
test_errors = []
for model, label in models:
    y_test_pred = model.predict(x_test)
    test_error = np.mean((y_test - y_test_pred) ** 2)
    test_errors.append(test_error)
    print(f"{label}: Test MSE = {test_error:.6f}")

print("=" * 50)
print(f"Noise variance σ² = {std**2:.6f}")

In [ ]:
# Visualize test errors
plt.figure(figsize=(10, 5))
labels = [label for _, label in models]
colors = ['green', 'red', 'orange', 'blue', 'purple']
bars = plt.bar(labels, test_errors, color=colors, alpha=0.7)
plt.axhline(y=std**2, color='black', linestyle='--', label=f'$\\sigma^2$ = {std**2}')
plt.xlabel('Model', fontsize=12)
plt.ylabel('Test MSE', fontsize=12)
plt.title('Test Error Comparison')
plt.legend()
plt.xticks(rotation=15)
plt.grid(True, alpha=0.3, axis='y')

# Add values on top of bars
for bar, error in zip(bars, test_errors):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005, 
             f'{error:.4f}', ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.show()

**Observations for part (e):**

The test error reveals the **generalization gap**:

1. **d=5, N=20** (correct model): Test error close to $\sigma^2$ - good generalization.
2. **d=20, N=20** (overfitting): Very high test error despite near-zero training error - severe overfitting.
3. **d=20, N≥50**: Test error decreases as N increases - more data reduces overfitting.
4. **d=20, N=5000**: Test error approaches that of the correct model (d=5).

The **expected test error** can be expressed as:
$$E_{\text{test}} = \sigma^2 + \sigma^2 \cdot \frac{d}{N} = \sigma^2 \left(1 + \frac{d}{N}\right)$$

This shows the variance component of the error, which decreases as $N/d$ increases.

---

# Additional Analysis

The following sections extend Exercise 1 to deepen our understanding of linear regression concepts.

## 1. Bias-Variance Tradeoff Analysis

The test error can be decomposed as:
$$E_{\text{test}} = \text{Bias}^2 + \text{Variance} + \sigma^2$$

We'll empirically estimate these components by running multiple experiments.

In [ ]:
def compute_bias_variance(d, N, n_experiments=100, n_test=500):
    """Estimate bias and variance empirically"""
    x_test = np.linspace(-1, 1, n_test)
    y_true = gt_d5.f(x_test)
    
    predictions = np.zeros((n_experiments, n_test))
    
    for i in range(n_experiments):
        x_train, y_train = generate_samples(gt_d5.f, N=N, mu=mu, std=std)
        model = LegendreRegression(d=d)
        model.fit(x_train, y_train)
        predictions[i] = model.predict(x_test)
    
    # Mean prediction
    mean_pred = np.mean(predictions, axis=0)
    
    # Bias^2 = E[(E[f_hat] - f_true)^2]
    bias_squared = np.mean((mean_pred - y_true) ** 2)
    
    # Variance = E[(f_hat - E[f_hat])^2]
    variance = np.mean(np.var(predictions, axis=0))
    
    return bias_squared, variance

# Analyze for different d values
d_values = [1, 3, 5, 10, 15, 20]
N_fixed = 50
bias_list, var_list = [], []

for d_val in d_values:
    b, v = compute_bias_variance(d_val, N_fixed, n_experiments=50)
    bias_list.append(b)
    var_list.append(v)
    print(f"d={d_val:2d}: Bias²={b:.5f}, Variance={v:.5f}, Total={b+v+std**2:.5f}")

In [ ]:
# Plot bias-variance tradeoff
plt.figure(figsize=(10, 6))
plt.plot(d_values, bias_list, 'b-o', label='Bias²', linewidth=2, markersize=8)
plt.plot(d_values, var_list, 'r-o', label='Variance', linewidth=2, markersize=8)
plt.plot(d_values, [b+v for b, v in zip(bias_list, var_list)], 'g--o', 
         label='Bias² + Variance', linewidth=2, markersize=8)
plt.axhline(y=std**2, color='gray', linestyle=':', label=f'$\\sigma^2$ = {std**2}')
plt.axvline(x=5, color='orange', linestyle='--', alpha=0.7, label='True d=5')
plt.xlabel('Model Complexity (d)', fontsize=12)
plt.ylabel('Error Component', fontsize=12)
plt.title('Bias-Variance Tradeoff (N=50)', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. K-Fold Cross-Validation

Cross-validation provides a more robust estimate of test error when we can't afford a separate test set. We use K-Fold CV to select the optimal model complexity $d$.

$$\text{CV}_K = \frac{1}{K} \sum_{k=1}^{K} \text{MSE}_k$$

In [ ]:
def k_fold_cv(x, y, d, K=5):
    """Perform K-fold cross-validation"""
    n = len(x)
    indices = np.arange(n)
    np.random.shuffle(indices)
    fold_size = n // K
    
    cv_errors = []
    for k in range(K):
        # Split data
        val_idx = indices[k*fold_size : (k+1)*fold_size]
        train_idx = np.concatenate([indices[:k*fold_size], indices[(k+1)*fold_size:]])
        
        x_train, y_train = x[train_idx], y[train_idx]
        x_val, y_val = x[val_idx], y[val_idx]
        
        # Fit and evaluate
        model = LegendreRegression(d=d)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_val)
        cv_errors.append(np.mean((y_val - y_pred) ** 2))
    
    return np.mean(cv_errors), np.std(cv_errors)

# Use N=100 dataset for CV
x_cv, y_cv = generate_samples(gt_d5.f, N=100, mu=mu, std=std)

# CV for different d values
d_range = range(1, 25)
cv_means, cv_stds = [], []

for d_val in d_range:
    mean, std_cv = k_fold_cv(x_cv, y_cv, d_val, K=5)
    cv_means.append(mean)
    cv_stds.append(std_cv)

optimal_d = d_range[np.argmin(cv_means)]
print(f"Optimal d from 5-fold CV: {optimal_d}")
print(f"CV Error at optimal d: {cv_means[optimal_d-1]:.5f}")

In [ ]:
# Plot CV curve
plt.figure(figsize=(10, 6))
plt.errorbar(list(d_range), cv_means, yerr=cv_stds, fmt='b-o', capsize=3, 
             linewidth=2, markersize=6, label='5-Fold CV Error')
plt.axvline(x=optimal_d, color='red', linestyle='--', label=f'Optimal d = {optimal_d}')
plt.axvline(x=5, color='green', linestyle=':', label='True d = 5')
plt.xlabel('Model Complexity (d)', fontsize=12)
plt.ylabel('Cross-Validation MSE', fontsize=12)
plt.title('Model Selection via K-Fold Cross-Validation', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Ridge Regularization (L2 Penalty)

Ridge regression adds an L2 penalty to control model complexity:
$$\hat{\theta}_{\text{ridge}} = \arg\min_\theta \|y - X\theta\|_2^2 + \lambda \|\theta\|_2^2$$

The closed-form solution is:
$$\hat{\theta}_{\text{ridge}} = (X^T X + \lambda I)^{-1} X^T y$$

In [ ]:
class RidgeRegression:
    """Legendre polynomial regression with L2 regularization"""
    def __init__(self, d=5, lam=0.1):
        self.d = d
        self.lam = lam
        self.L_vec = [legendre(i) for i in range(d)]
        self.theta = np.zeros(d)

    def fit(self, x, y):
        X = np.column_stack([self.L_vec[i](x) for i in range(self.d)])
        self.theta = np.linalg.inv(X.T @ X + self.lam * np.eye(self.d)) @ X.T @ y
        return self

    def predict(self, x):
        y_hat = self.theta[0] * self.L_vec[0](x)
        for i in range(1, self.d):
            y_hat += self.theta[i] * self.L_vec[i](x)
        return y_hat

# Test with d=20, N=20 (overfitting case)
lambdas = np.logspace(-4, 2, 30)
train_errors_ridge = []
test_errors_ridge = []
coef_norms = []

for lam in lambdas:
    model = RidgeRegression(d=20, lam=lam)
    model.fit(x_N20, y_N20)
    
    train_errors_ridge.append(np.mean((y_N20 - model.predict(x_N20)) ** 2))
    test_errors_ridge.append(np.mean((y_test - model.predict(x_test)) ** 2))
    coef_norms.append(np.linalg.norm(model.theta))

optimal_lam = lambdas[np.argmin(test_errors_ridge)]
print(f"Optimal λ: {optimal_lam:.4f}")
print(f"Test error at optimal λ: {min(test_errors_ridge):.5f}")
print(f"Test error without regularization (d=20, N=20): {test_errors[1]:.5f}")

In [ ]:
# Plot Ridge regression results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Error vs lambda
axes[0].semilogx(lambdas, train_errors_ridge, 'b-', label='Train Error', linewidth=2)
axes[0].semilogx(lambdas, test_errors_ridge, 'r-', label='Test Error', linewidth=2)
axes[0].axvline(x=optimal_lam, color='green', linestyle='--', label=f'Optimal λ = {optimal_lam:.4f}')
axes[0].axhline(y=std**2, color='gray', linestyle=':', label=f'$\\sigma^2$ = {std**2}')
axes[0].set_xlabel('λ (regularization)', fontsize=12)
axes[0].set_ylabel('MSE', fontsize=12)
axes[0].set_title('Ridge Regression: Error vs Regularization', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Right: Coefficient norm vs lambda
axes[1].semilogx(lambdas, coef_norms, 'purple', linewidth=2)
axes[1].axvline(x=optimal_lam, color='green', linestyle='--', label=f'Optimal λ')
axes[1].set_xlabel('λ (regularization)', fontsize=12)
axes[1].set_ylabel('||θ||₂', fontsize=12)
axes[1].set_title('Coefficient Shrinkage', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Numerical Conditioning Analysis

The condition number of $X^TX$ determines numerical stability:
$$\kappa(X^TX) = \frac{\sigma_{\max}(X^TX)}{\sigma_{\min}(X^TX)}$$

A high condition number indicates the problem is ill-conditioned and solutions may be unstable.

In [ ]:
def compute_condition_number(x, d):
    """Compute condition number of X^T X"""
    L_vec = [legendre(i) for i in range(d)]
    X = np.column_stack([L_vec[i](x) for i in range(d)])
    XTX = X.T @ X
    return np.linalg.cond(XTX)

# Analyze condition numbers for different scenarios
print("Condition Numbers:")
print("=" * 50)

scenarios = [
    (x_N20, 5, "d=5, N=20"),
    (x_N20, 20, "d=20, N=20"),
    (x_N50, 20, "d=20, N=50"),
    (x_N500, 20, "d=20, N=500"),
    (x_N5000, 20, "d=20, N=5000"),
]

cond_numbers = []
for x, d_val, label in scenarios:
    cond = compute_condition_number(x, d_val)
    cond_numbers.append(cond)
    print(f"{label}: κ(X^TX) = {cond:.2e}")

print("=" * 50)
print("Note: Condition number < 10⁶ is typically considered acceptable")

## 5. Training vs Test Error Curve

This classic visualization shows how training and test errors diverge as model complexity increases.

- **Underfitting** (low d): High bias, both training and test error are high
- **Sweet spot** (optimal d): Low bias and variance, test error is minimized
- **Overfitting** (high d): High variance, training error is low but test error is high

In [ ]:
# Generate training vs test error curve
N_train = 100
x_train_curve, y_train_curve = generate_samples(gt_d5.f, N=N_train, mu=mu, std=std)
x_test_curve, y_test_curve = generate_samples(gt_d5.f, N=1000, mu=mu, std=std)

d_range_full = range(1, 40)
train_errors_curve = []
test_errors_curve = []

for d_val in d_range_full:
    model = LegendreRegression(d=d_val)
    model.fit(x_train_curve, y_train_curve)
    
    train_pred = model.predict(x_train_curve)
    test_pred = model.predict(x_test_curve)
    
    train_errors_curve.append(np.mean((y_train_curve - train_pred) ** 2))
    test_errors_curve.append(np.mean((y_test_curve - test_pred) ** 2))

# Plot
plt.figure(figsize=(10, 6))
plt.plot(list(d_range_full), train_errors_curve, 'b-o', label='Training Error', 
         linewidth=2, markersize=4)
plt.plot(list(d_range_full), test_errors_curve, 'r-o', label='Test Error', 
         linewidth=2, markersize=4)
plt.axvline(x=5, color='green', linestyle='--', alpha=0.7, label='True d=5')
plt.axhline(y=std**2, color='gray', linestyle=':', label=f'$\\sigma^2$ = {std**2}')
plt.xlabel('Model Complexity (d)', fontsize=12)
plt.ylabel('MSE', fontsize=12)
plt.title(f'Training vs Test Error (N={N_train})', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim([0, 0.5])
plt.show()

optimal_d_test = d_range_full[np.argmin(test_errors_curve)]
print(f"Optimal d (minimum test error): {optimal_d_test}")

## 6. Coefficient Analysis

Comparing learned coefficients with true coefficients helps us understand how well the model recovers the ground truth parameters.

In [ ]:
# Compare coefficients for d=5 models with different N
fig, ax = plt.subplots(figsize=(10, 6))

# Fit models with d=5 for different N
coef_data = {}
for N_val, x_data, y_data in [(20, x_N20, y_N20), (50, x_N50, y_N50), 
                               (500, x_N500, y_N500), (5000, x_N5000, y_N5000)]:
    model = LegendreRegression(d=5)
    model.fit(x_data, y_data)
    coef_data[N_val] = model.theta

# Plot
x_pos = np.arange(5)
width = 0.15
multiplier = 0

ax.bar(x_pos - 2*width, theta, width, label='True θ', color='black', alpha=0.8)
for N_val, coefs in coef_data.items():
    offset = width * multiplier
    ax.bar(x_pos + offset - width, coefs, width, label=f'N={N_val}', alpha=0.7)
    multiplier += 1

ax.set_xlabel('Coefficient Index', fontsize=12)
ax.set_ylabel('Coefficient Value', fontsize=12)
ax.set_title('Learned Coefficients vs True Coefficients (d=5)', fontsize=14)
ax.set_xticks(x_pos)
ax.set_xticklabels([f'θ₀', f'θ₁', f'θ₂', f'θ₃', f'θ₄'])
ax.legend()
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

# Print numerical comparison
print("\nCoefficient Comparison:")
print("=" * 60)
print(f"{'Coef':<6} {'True':<10} {'N=20':<10} {'N=50':<10} {'N=500':<10} {'N=5000':<10}")
print("=" * 60)
for i in range(5):
    print(f"θ_{i:<4} {theta[i]:<10.4f} {coef_data[20][i]:<10.4f} {coef_data[50][i]:<10.4f} "
          f"{coef_data[500][i]:<10.4f} {coef_data[5000][i]:<10.4f}")

## 7. Learning Curve

Learning curves show how training and test errors evolve as the training set size increases. This helps determine:
- Whether collecting more data would help
- If the model is limited by high bias (underfitting) or high variance (overfitting)

In [ ]:
# Learning curve for d=5 and d=20
def compute_learning_curve(d, N_values, n_experiments=30):
    """Compute average train and test errors for different N"""
    train_means, train_stds = [], []
    test_means, test_stds = [], []
    
    for N in N_values:
        train_errs, test_errs = [], []
        for _ in range(n_experiments):
            x_train, y_train = generate_samples(gt_d5.f, N=N, mu=mu, std=std)
            x_test_lc, y_test_lc = generate_samples(gt_d5.f, N=500, mu=mu, std=std)
            
            model = LegendreRegression(d=d)
            model.fit(x_train, y_train)
            
            train_errs.append(np.mean((y_train - model.predict(x_train)) ** 2))
            test_errs.append(np.mean((y_test_lc - model.predict(x_test_lc)) ** 2))
        
        train_means.append(np.mean(train_errs))
        train_stds.append(np.std(train_errs))
        test_means.append(np.mean(test_errs))
        test_stds.append(np.std(test_errs))
    
    return train_means, train_stds, test_means, test_stds

N_values = [10, 20, 30, 50, 75, 100, 150, 200, 300, 500]

# Compute learning curves
lc_d5 = compute_learning_curve(5, N_values, n_experiments=20)
lc_d20 = compute_learning_curve(20, N_values, n_experiments=20)

In [ ]:
# Plot learning curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# d=5 (correct model)
axes[0].fill_between(N_values, 
                      np.array(lc_d5[0]) - np.array(lc_d5[1]), 
                      np.array(lc_d5[0]) + np.array(lc_d5[1]), alpha=0.2, color='blue')
axes[0].fill_between(N_values, 
                      np.array(lc_d5[2]) - np.array(lc_d5[3]), 
                      np.array(lc_d5[2]) + np.array(lc_d5[3]), alpha=0.2, color='red')
axes[0].plot(N_values, lc_d5[0], 'b-o', label='Train Error', linewidth=2, markersize=6)
axes[0].plot(N_values, lc_d5[2], 'r-o', label='Test Error', linewidth=2, markersize=6)
axes[0].axhline(y=std**2, color='gray', linestyle=':', label=f'$\\sigma^2$')
axes[0].set_xlabel('Training Set Size (N)', fontsize=12)
axes[0].set_ylabel('MSE', fontsize=12)
axes[0].set_title('Learning Curve: d=5 (Correct Model)', fontsize=14)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# d=20 (complex model)
axes[1].fill_between(N_values, 
                      np.array(lc_d20[0]) - np.array(lc_d20[1]), 
                      np.array(lc_d20[0]) + np.array(lc_d20[1]), alpha=0.2, color='blue')
axes[1].fill_between(N_values, 
                      np.array(lc_d20[2]) - np.array(lc_d20[3]), 
                      np.array(lc_d20[2]) + np.array(lc_d20[3]), alpha=0.2, color='red')
axes[1].plot(N_values, lc_d20[0], 'b-o', label='Train Error', linewidth=2, markersize=6)
axes[1].plot(N_values, lc_d20[2], 'r-o', label='Test Error', linewidth=2, markersize=6)
axes[1].axhline(y=std**2, color='gray', linestyle=':', label=f'$\\sigma^2$')
axes[1].set_xlabel('Training Set Size (N)', fontsize=12)
axes[1].set_ylabel('MSE', fontsize=12)
axes[1].set_title('Learning Curve: d=20 (Complex Model)', fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Summary

This notebook explored linear regression using Legendre polynomials, demonstrating:

1. **Overfitting**: When $d \geq N$, the model memorizes training data but fails to generalize.

2. **Bias-Variance Tradeoff**: 
   - Low $d$: High bias, low variance (underfitting)
   - High $d$: Low bias, high variance (overfitting)
   - Optimal $d$: Balances both components

3. **More Data Helps**: Increasing $N$ reduces overfitting even for complex models.

4. **Training Error Formula**: $E_{\text{train}} \approx \sigma^2(1 - d/N)$

5. **Test Error Formula**: $E_{\text{test}} \approx \sigma^2(1 + d/N)$

6. **Regularization**: Ridge regression (L2 penalty) effectively controls overfitting.

7. **Cross-Validation**: K-fold CV provides robust model selection without a separate test set.

### Key Takeaways

- The **ratio $d/N$** is critical: keep it small for good generalization
- **Model selection** (choosing $d$) is as important as model fitting
- **Regularization** provides an alternative to reducing model complexity
- **Learning curves** help diagnose underfitting vs overfitting